In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Libraries**

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.4 MB/s eta 0:00:00


In [ ]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.9 MB/s eta 0:00:00


**Imports**

In [ ]:
import pandas as pd
import tiktoken
import time

In [ ]:

def token_count(text):

  encoding = tiktoken.get_encoding("cl100k_base")
  # encoding = tiktoken.get_encoding("p50k_base")
  # print(encoding)
  # print(encoding.encode(text))
  # print(type(encoding.encode(text)))
  num_tokens = len(encoding.encode(text))
  return num_tokens

In [ ]:
def convert_response(response):
  sentiment=response['choices'][0]["message"]['content']
  sentiment=sentiment.replace("Apple -","").replace("Target 1","Apple")
  sentiment=sentiment.replace("Samsung - ","").replace("Target 2","Samsung")
  lines = sentiment.split('\n')
  dictionary = {}
  for line in lines:
      key, value = line.split(': ')
      target, sentiment = key.strip(), value.strip()
      dictionary[target] = sentiment
      print("***********************************************")
      print(dictionary)
      print(len(dictionary.keys()))
      print("***********************************************")
  return dictionary

In [ ]:
def convert_response1(response):
  sentiment=response['choices'][0]["message"]['content']

  lines = sentiment.split('\n')
  dictionary = {}
  for line in lines:
      if "Target 1" in line or "Apple" in line:
        print("اپل موجوده ")
        if 'Neutral' in line:
          dictionary["Apple"]="Neutral"
        if "Disagree" in line:
          dictionary["Apple"]="Disagree"
        if "Agree" in line:
          dictionary["Apple"]="Agree"
      else:
        print("اپل موجود نیست")
      if "Target 2" in line or "Samsung" in line:
        print("سامسونگ موجوده ")
        if 'Neutral' in line:
          dictionary["Samsung"]="Neutral"
        if "Disagree" in line:
          dictionary["Samsung"]="Disagree"
        if "Agree" in line:
          dictionary["Samsung"]="Agree"
      else:
        print("سامسونگ موجود نیست")
  return dictionary

In [ ]:
from numpy.core.multiarray import promote_types
import openai
import pandas as pd
import time
# Set up your OpenAI API key
openai.api_key = "your apikey"
prompt='''Label the sentiment of the following text towards Apple and Samsung. Provide two labels, one for each target:

Target 1: Apple
Target 2: Samsung

For each target, choose one of the following labels:
- Agree: If the text expresses a positive sentiment towards the target.
- Disagree: If the text expresses a negative sentiment towards the target.
- Neutral: If the text does not express any sentiment towards the target.
Text : {}
'''

def analyze_sentiments(text):
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[ {"role": "user", "content": prompt.format(text)}, ]
    )
    time.sleep(60)
    return response


In [ ]:

df=pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/ui2/subset2.csv")
def run_code():

  for i, row in df.iterrows():

        samsung=pd.isna(df.loc[i, "samsung"])
        apple=pd.isna(df.loc[i, "apple"])
        print(samsung,apple)
        if samsung==True and apple==True:
          print(i)
          text = row["comments"]
          count=token_count(text)
          # print(count)
          print(text)
          print("/////////////////////////////////////////////")
          if count<=1900:
            df.loc[i, 'token'] = count
            # if i<=10:
            print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< start calling api <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
            response=analyze_sentiments(text)
            final_response=convert_response1(response)
            print(final_response)
            if len(final_response.keys())==2:
              print("samsung is : ",final_response['Samsung'])
              print("apple is : ",final_response['Apple'])
              df.loc[i, 'samsung'] = final_response['Samsung']
              df.loc[i, 'apple'] = final_response['Apple']
              df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/ui2/subset2.csv", index=False)
              print(" bro usage ro negah kon")
              print("start sleep")
              print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
              # time.sleep(60)
            else:
              print("#########################################################################")
              print(final_response.keys())
              print("#########################################################################")

            # df['token']=count
          else:
            # df['token']="no"
            df.loc[i, 'token'] = "no"
            df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/ui2/subset2.csv", index=False)


In [ ]:
run_code()

True True
0
سلام دوستان وقتتون بخیر ممنون میشم دوستانی که آیفون X دارن نظرشونو درباره باطریش بگن آیا یه روز استفاده معمولی رو جواب میده یا ن؟
/////////////////////////////////////////////
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< start calling api <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
اپل موجوده 
سامسونگ موجود نیست
اپل موجود نیست
سامسونگ موجوده 
{'Apple': 'Neutral', 'Samsung': 'Neutral'}
samsung is :  Neutral
apple is :  Neutral
 bro usage ro negah kon
start sleep
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
True True
1
» در پاسخ به نظر                                                     Mtiinw M                                                به تاریخ                         1401/4/22                        (برای نمایش نظر کلیک کنید...)                                                سلام واقعیتش آیفون ایکس معمولی رو نگیری بهتره بخاطر اینکه همش دست دوم هستند تو بازار ،میتونی تو سایت gsmarena  نگاه کنی حتی تو  آم

KeyboardInterrupt: ignored

In [ ]:
def split_dataset():

  sheet_id="1GG6dw2tVSb9Sp8jrnuy-FdjPP_JYdt5K7OoN_32j6gE"
  df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")



  # Calculate the number of rows in each subset
  n = len(df)
  n1 = int(n/3)  # subset 1 will have 1/3 of the rows
  n2 = int(n/3)  # subset 2 will have 1/3 of the rows
  n3 = n - n1 - n2  # subset 3 will have the remaining rows

  # Create the three subsets based on the calculated row counts
  subset1 = df[:n1]
  subset2 = df[n1:n1+n2]
  subset3 = df[n1+n2:]

  subset1= subset1.assign(token="")
  subset2= subset2.assign(token="")
  subset3= subset3.assign(token="")
  # subset1["token"]=""
  # subset2["token"]=""
  # subset3["token"]=""
  # print(type(subset1))
  # Write each subset to a new CSV file
  subset1.to_csv("/content/gdrive/MyDrive/Colab Notebooks/ui2/subset1.csv", index=False)
  subset2.to_csv("/content/gdrive/MyDrive/Colab Notebooks/ui2/subset2.csv", index=False)
  subset3.to_csv("/content/gdrive/MyDrive/Colab Notebooks/ui2/subset3.csv", index=False)

In [ ]:
split_dataset()